In [6]:
# Wed 17:10 
# Creates rectangle "depth images", a CNN, solves, and graphs the losses (by epoch)
# as well as the labels against the original image, and the true image (with x's)
# nrw
from PIL import Image, ImageDraw, ImageFont
%matplotlib inline
import numpy as np

import torch
import torch.nn as nn
from skimage import io
import math

from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

import matplotlib.pyplot as plt
from IPython.display import Audio


In [2]:

IMG_X, IMG_Y = 400,300
# length and width of blocks (fixed for now)
block_l, block_w = 25, 25
num_images = 1500

# img_list = []
true_coords = []


# Calc rectangle vertices. makeRectangle() credit Sparkler, stackoverflow, feb 17
def makeRectangle(l, w, theta, offset=(0, 0)):
    c, s = math.cos(theta), math.sin(theta)
    rectCoords = [(l/2.0, w/2.0), (l/2.0, -w/2.0), (-l/2.0, -w/2.0), (-l/2.0, w/2.0)]
    return [(c*x-s*y+offset[0], s*x+c*y+offset[1]) for (x, y) in rectCoords]


# ---- Make depth images ---
for i in range(num_images):
    orient = 0 # degrees
    img = Image.new('RGB', (IMG_X, IMG_Y), 'black')

    # block_l and _w offset so blocks don't run off edge of image
    rand_x = int(np.random.rand() * (IMG_X-block_l))
    rand_y = int(np.random.rand() * (IMG_Y-block_w))

    true_coords.append(np.array((rand_x, rand_y)))

    rect_vertices = makeRectangle(block_l, block_w, orient, offset=(rand_x,
                                                                    rand_y))
    idraw = ImageDraw.Draw(img)
    idraw.polygon(rect_vertices, fill='white')

    # use a truetype font
    #font = ImageFont.truetype("DejavuSans.ttf", 15)
    #font = ImageFont.truetype("Arial.ttf",14)
    #idraw.text((10, 25), '('+ str(rand_x) + ', ' + str(rand_y) +')')


    img.save('./data/rect'+str(i)+'.png')


    
class RectDepthImgsDataset(Dataset):
    """Artificially generated depth images dataset"""

    def __init__(self, img_dir, coords, transform=None):
        """
        """
        self.img_dir = img_dir
        self.true_coords = coords
        self.transform = transform

    def __len__(self):
        return len(self.true_coords)

    def __getitem__(self, idx):
        # image = self.images[idx]
        image = io.imread(self.img_dir + '/rect'+str(idx)+'.png')
        image = torch.FloatTensor(image).permute(2, 0, 1) #PIL and torch expect difft orders
        coords = torch.FloatTensor(true_coords[idx])

        if self.transform:
            image = self.transform(image)

        # sample = {'image': image, 'grasp': str(coords[0]) + str(coords[1])}
        sample = {'image': image, 'grasp': coords}
        sample = image, coords

        return sample

In [3]:
# Hyper parameters
num_epochs = 50 
num_classes = 2
batch_size = 4 
learning_rate = 0.001

# Dataset is depth images of rectangular blocks
train_dataset = RectDepthImgsDataset(img_dir='./data', coords=true_coords)

# Data loader
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size,
                          shuffle=True)

const = 16*47*47
#const = 16*97*72


# 300x300
#torch.Size([4, 300, 300, 3])
#torch.Size([4, 3, 300, 300])
#torch.Size([4, 6, 148, 148])
#torch.Size([4, 16, 72, 72])


# 200x200
#const = 16*47*47*4
#torch.Size([4, 200, 200, 3])
#torch.Size([4, 3, 200, 200])
#torch.Size([4, 6, 98, 98])
#torch.Size([4, 16, 47, 47])

# = 200-4 / 2
# where 4 = 5x5 (conv) -1 = area of image after conv without padding = 200->98
# = 98-4 / 2
# where 4 = 5x5 (conv) -1 = area of image after conv without padding = 98->47
# output is 47x47 image, with 16 output channels


# 400x300
#torch.Size([4, 300, 400, 3])
#torch.Size([4, 3, 400, 300])
#torch.Size([4, 6, 198, 148])
#torch.Size([4, 16, 97, 72])


class Net(nn.Module):  # CIFAR is 32x32x3, MNIST is 28x28x1)
    def __init__(self, IMG_X, IMG_Y):
        super(Net, self).__init__()
        self._imgx = IMG_X
        self._imgy = IMG_Y
        _pool = 2
        _stride = 5
        _outputlayers = 16
        
        def _calc(val):
            layer_size = (val- (_stride-1)) / _pool
            return layer_size 
        
        #print(self._imgx)
        self._const = _calc(_calc(self._imgx))
        self._const *= _calc(_calc(self._imgy))
        self._const *= _outputlayers 
        #print(self._const)
        self._const = int(self._const)
        
        self.conv1 = nn.Conv2d(3, 6, _stride)
        self.pool = nn.MaxPool2d(_pool, _pool)
        self.conv2 = nn.Conv2d(6, _outputlayers, _stride)
        self.fc1 = nn.Linear(self._const, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)
        

    def forward(self, x):
        #print(x.size())
        x = x.view(-1, 3, IMG_X, IMG_Y)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, self._const)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = Net(IMG_X, IMG_Y)
images = iter(train_loader)
outputs = model(images.next()[0])

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [4]:
losses_list = []
print('Training model now...')
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i_batch, (images, labels) in enumerate(train_loader):

        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        loss.backward()
        optimizer.step()

        if (i_batch+1) % 1 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1,
                                                                      num_epochs,
                                                                      i_batch+1,
                                                                      total_step,
                                                                      loss.item()))
            losses_list.append(loss.item())

Training model now...
Epoch [1/50], Step [1/375], Loss: 52234.1133
Epoch [1/50], Step [2/375], Loss: 19984.1602
Epoch [1/50], Step [3/375], Loss: 22110.1738
Epoch [1/50], Step [4/375], Loss: 16244.5195
Epoch [1/50], Step [5/375], Loss: 7527.7583
Epoch [1/50], Step [6/375], Loss: 5610.8462
Epoch [1/50], Step [7/375], Loss: 44593.1523
Epoch [1/50], Step [8/375], Loss: 15119.0801
Epoch [1/50], Step [9/375], Loss: 23256.7695
Epoch [1/50], Step [10/375], Loss: 12390.0557
Epoch [1/50], Step [11/375], Loss: 12360.8604
Epoch [1/50], Step [12/375], Loss: 30499.1875
Epoch [1/50], Step [13/375], Loss: 3690.3381
Epoch [1/50], Step [14/375], Loss: 10630.9932
Epoch [1/50], Step [15/375], Loss: 13804.3877
Epoch [1/50], Step [16/375], Loss: 7968.5840
Epoch [1/50], Step [17/375], Loss: 15270.2881
Epoch [1/50], Step [18/375], Loss: 41949.2461
Epoch [1/50], Step [19/375], Loss: 25963.5371
Epoch [1/50], Step [20/375], Loss: 27096.8223
Epoch [1/50], Step [21/375], Loss: 15464.5664
Epoch [1/50], Step [22/37

Epoch [1/50], Step [181/375], Loss: 6078.9692
Epoch [1/50], Step [182/375], Loss: 7690.0483
Epoch [1/50], Step [183/375], Loss: 8498.9717
Epoch [1/50], Step [184/375], Loss: 10774.4990
Epoch [1/50], Step [185/375], Loss: 5250.1016
Epoch [1/50], Step [186/375], Loss: 5597.1860
Epoch [1/50], Step [187/375], Loss: 6868.5830
Epoch [1/50], Step [188/375], Loss: 7371.2456
Epoch [1/50], Step [189/375], Loss: 1560.0724
Epoch [1/50], Step [190/375], Loss: 6739.6270
Epoch [1/50], Step [191/375], Loss: 1634.0278
Epoch [1/50], Step [192/375], Loss: 2716.3025
Epoch [1/50], Step [193/375], Loss: 4871.0093
Epoch [1/50], Step [194/375], Loss: 6763.3110
Epoch [1/50], Step [195/375], Loss: 1857.4230
Epoch [1/50], Step [196/375], Loss: 7994.8701
Epoch [1/50], Step [197/375], Loss: 1385.5677
Epoch [1/50], Step [198/375], Loss: 5667.6362
Epoch [1/50], Step [199/375], Loss: 6237.8159
Epoch [1/50], Step [200/375], Loss: 6258.0469
Epoch [1/50], Step [201/375], Loss: 2089.6096
Epoch [1/50], Step [202/375], Los

Epoch [1/50], Step [359/375], Loss: 7903.2666
Epoch [1/50], Step [360/375], Loss: 4100.9961
Epoch [1/50], Step [361/375], Loss: 7670.7969
Epoch [1/50], Step [362/375], Loss: 6008.6909
Epoch [1/50], Step [363/375], Loss: 10513.2725
Epoch [1/50], Step [364/375], Loss: 7117.0562
Epoch [1/50], Step [365/375], Loss: 9298.4678
Epoch [1/50], Step [366/375], Loss: 4316.3442
Epoch [1/50], Step [367/375], Loss: 2588.3147
Epoch [1/50], Step [368/375], Loss: 4523.0483
Epoch [1/50], Step [369/375], Loss: 13003.1738
Epoch [1/50], Step [370/375], Loss: 3681.2712
Epoch [1/50], Step [371/375], Loss: 7388.8281
Epoch [1/50], Step [372/375], Loss: 12293.6621
Epoch [1/50], Step [373/375], Loss: 8175.7041
Epoch [1/50], Step [374/375], Loss: 14953.7314
Epoch [1/50], Step [375/375], Loss: 3519.2703
Epoch [2/50], Step [1/375], Loss: 8668.6709
Epoch [2/50], Step [2/375], Loss: 4071.4028
Epoch [2/50], Step [3/375], Loss: 3575.3765
Epoch [2/50], Step [4/375], Loss: 4599.8350
Epoch [2/50], Step [5/375], Loss: 8287

Epoch [2/50], Step [166/375], Loss: 2472.8000
Epoch [2/50], Step [167/375], Loss: 5523.2026
Epoch [2/50], Step [168/375], Loss: 5454.5923
Epoch [2/50], Step [169/375], Loss: 4378.1230
Epoch [2/50], Step [170/375], Loss: 8133.5732
Epoch [2/50], Step [171/375], Loss: 2207.1880
Epoch [2/50], Step [172/375], Loss: 5228.5596
Epoch [2/50], Step [173/375], Loss: 4979.1494
Epoch [2/50], Step [174/375], Loss: 2485.0310
Epoch [2/50], Step [175/375], Loss: 5228.2812
Epoch [2/50], Step [176/375], Loss: 4851.5225
Epoch [2/50], Step [177/375], Loss: 3736.1882
Epoch [2/50], Step [178/375], Loss: 7289.9834
Epoch [2/50], Step [179/375], Loss: 2790.8867
Epoch [2/50], Step [180/375], Loss: 2768.8286
Epoch [2/50], Step [181/375], Loss: 3769.9182
Epoch [2/50], Step [182/375], Loss: 3846.4614
Epoch [2/50], Step [183/375], Loss: 6446.9434
Epoch [2/50], Step [184/375], Loss: 2841.7585
Epoch [2/50], Step [185/375], Loss: 1010.3520
Epoch [2/50], Step [186/375], Loss: 3843.8831
Epoch [2/50], Step [187/375], Loss

Epoch [2/50], Step [346/375], Loss: 3598.3057
Epoch [2/50], Step [347/375], Loss: 1865.8590
Epoch [2/50], Step [348/375], Loss: 2554.5625
Epoch [2/50], Step [349/375], Loss: 5502.9331
Epoch [2/50], Step [350/375], Loss: 5218.9307
Epoch [2/50], Step [351/375], Loss: 2773.1475
Epoch [2/50], Step [352/375], Loss: 2185.9585
Epoch [2/50], Step [353/375], Loss: 5667.9883
Epoch [2/50], Step [354/375], Loss: 1620.7308
Epoch [2/50], Step [355/375], Loss: 4957.9956
Epoch [2/50], Step [356/375], Loss: 2725.2249
Epoch [2/50], Step [357/375], Loss: 2344.1211
Epoch [2/50], Step [358/375], Loss: 4552.4185
Epoch [2/50], Step [359/375], Loss: 4501.3535
Epoch [2/50], Step [360/375], Loss: 3553.4075
Epoch [2/50], Step [361/375], Loss: 2956.6658
Epoch [2/50], Step [362/375], Loss: 4895.9023
Epoch [2/50], Step [363/375], Loss: 4726.7510
Epoch [2/50], Step [364/375], Loss: 3854.8416
Epoch [2/50], Step [365/375], Loss: 4731.6270
Epoch [2/50], Step [366/375], Loss: 1919.2802
Epoch [2/50], Step [367/375], Loss

...

Epoch [18/50], Step [350/375], Loss: 192.7866
Epoch [18/50], Step [351/375], Loss: 186.3765
Epoch [18/50], Step [352/375], Loss: 119.4531
Epoch [18/50], Step [353/375], Loss: 367.7194
Epoch [18/50], Step [354/375], Loss: 294.3039
Epoch [18/50], Step [355/375], Loss: 117.0368
Epoch [18/50], Step [356/375], Loss: 158.2458
Epoch [18/50], Step [357/375], Loss: 131.8857
Epoch [18/50], Step [358/375], Loss: 53.4808
Epoch [18/50], Step [359/375], Loss: 203.0634
Epoch [18/50], Step [360/375], Loss: 118.2471
Epoch [18/50], Step [361/375], Loss: 468.8347
Epoch [18/50], Step [362/375], Loss: 130.4420
Epoch [18/50], Step [363/375], Loss: 263.1672
Epoch [18/50], Step [364/375], Loss: 43.7009
Epoch [18/50], Step [365/375], Loss: 100.5210
Epoch [18/50], Step [366/375], Loss: 41.2214
Epoch [18/50], Step [367/375], Loss: 440.7792
Epoch [18/50], Step [368/375], Loss: 405.5782
Epoch [18/50], Step [369/375], Loss: 115.0056
Epoch [18/50], Step [370/375], Loss: 188.0464
Epoch [18/50], Step [371/375], Loss: 

Epoch [19/50], Step [157/375], Loss: 129.1019
Epoch [19/50], Step [158/375], Loss: 210.2683
Epoch [19/50], Step [159/375], Loss: 259.0897
Epoch [19/50], Step [160/375], Loss: 159.3069
Epoch [19/50], Step [161/375], Loss: 160.3635
Epoch [19/50], Step [162/375], Loss: 80.1006
Epoch [19/50], Step [163/375], Loss: 229.6117
Epoch [19/50], Step [164/375], Loss: 314.9562
Epoch [19/50], Step [165/375], Loss: 40.8498
Epoch [19/50], Step [166/375], Loss: 220.5826
Epoch [19/50], Step [167/375], Loss: 96.1412
Epoch [19/50], Step [168/375], Loss: 237.5428
Epoch [19/50], Step [169/375], Loss: 57.4976
Epoch [19/50], Step [170/375], Loss: 148.9354
Epoch [19/50], Step [171/375], Loss: 418.5139
Epoch [19/50], Step [172/375], Loss: 163.3081
Epoch [19/50], Step [173/375], Loss: 336.9171
Epoch [19/50], Step [174/375], Loss: 89.1687
Epoch [19/50], Step [175/375], Loss: 586.7887
Epoch [19/50], Step [176/375], Loss: 124.9120
Epoch [19/50], Step [177/375], Loss: 432.0661
Epoch [19/50], Step [178/375], Loss: 38

Epoch [19/50], Step [337/375], Loss: 58.7956
Epoch [19/50], Step [338/375], Loss: 17.6785
Epoch [19/50], Step [339/375], Loss: 142.1848
Epoch [19/50], Step [340/375], Loss: 216.5814
Epoch [19/50], Step [341/375], Loss: 728.6064
Epoch [19/50], Step [342/375], Loss: 162.8519
Epoch [19/50], Step [343/375], Loss: 141.1270
Epoch [19/50], Step [344/375], Loss: 316.8925
Epoch [19/50], Step [345/375], Loss: 269.0276
Epoch [19/50], Step [346/375], Loss: 149.8856
Epoch [19/50], Step [347/375], Loss: 163.5600
Epoch [19/50], Step [348/375], Loss: 145.4756
Epoch [19/50], Step [349/375], Loss: 224.0555
Epoch [19/50], Step [350/375], Loss: 118.0933
Epoch [19/50], Step [351/375], Loss: 27.0484
Epoch [19/50], Step [352/375], Loss: 658.9689
Epoch [19/50], Step [353/375], Loss: 97.9224
Epoch [19/50], Step [354/375], Loss: 368.6955
Epoch [19/50], Step [355/375], Loss: 141.0322
Epoch [19/50], Step [356/375], Loss: 107.9864
Epoch [19/50], Step [357/375], Loss: 123.2555
Epoch [19/50], Step [358/375], Loss: 6

Epoch [20/50], Step [145/375], Loss: 192.1428
Epoch [20/50], Step [146/375], Loss: 705.6003
Epoch [20/50], Step [147/375], Loss: 77.4071
Epoch [20/50], Step [148/375], Loss: 41.3133
Epoch [20/50], Step [149/375], Loss: 163.5903
Epoch [20/50], Step [150/375], Loss: 199.6785
Epoch [20/50], Step [151/375], Loss: 179.1439
Epoch [20/50], Step [152/375], Loss: 81.4574
Epoch [20/50], Step [153/375], Loss: 108.0773
Epoch [20/50], Step [154/375], Loss: 146.9574
Epoch [20/50], Step [155/375], Loss: 90.8368
Epoch [20/50], Step [156/375], Loss: 60.0402
Epoch [20/50], Step [157/375], Loss: 84.8998
Epoch [20/50], Step [158/375], Loss: 85.2680
Epoch [20/50], Step [159/375], Loss: 29.4550
Epoch [20/50], Step [160/375], Loss: 122.9722
Epoch [20/50], Step [161/375], Loss: 67.3295
Epoch [20/50], Step [162/375], Loss: 140.4646
Epoch [20/50], Step [163/375], Loss: 219.2854
Epoch [20/50], Step [164/375], Loss: 231.7207
Epoch [20/50], Step [165/375], Loss: 121.4518
Epoch [20/50], Step [166/375], Loss: 44.311

FileNotFoundError: [Errno 2] No such file or directory: './data/rect1052.png'

In [ ]:
from IPython.display import display # to display images
def imshow_coord(img, a_label):
    img = torchvision.transforms.ToPILImage()(img)
    draw = ImageDraw.Draw(img)
    draw = draw_crosspointer(a_label, draw, 'green', 'white', length=8)
    display(img)
    
def draw_crosspointer(xy, draw, fillcolor_X='green', fillcolor='white', length=2):
    a,b = tuple(xy)
    draw.line((a-length, b+length, a+length, b-length), fill=fillcolor_X)
    draw.line((a-length, b-length, a+length, b+length), fill=fillcolor_X)
    draw.point((a,b))
    return draw

In [7]:
print('All ready!')
# alert when training is done
sound_file = '/home/rui/Downloads/newyear.ogg'
Audio(sound_file, autoplay=True)

All ready!


In [ ]:
import torchvision 
import matplotlib.pyplot as plt
import copy
# get some random training images
with torch.no_grad():
    dataiter = iter(train_loader)
    images, labels = dataiter.next()
    outputs = model(images)
    
    # show images
    #imshow_coord(torchvision.utils.make_grid(images), (outputs))


    # print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                                  # for j in range(4)))
 
n =1 
print(len(labels))
print('x,y truth', labels[n])
print('x,y guess by net', outputs[n], '\n')

images_copy = copy.deepcopy(images)
print('image size: ', images_copy[n].size(), '\n')
print('net output: ')
imshow_coord(images_copy[n], outputs[n])
print('truth label: ')
imshow_coord(images_copy[n], labels[n])

#imagePIL = torchvision.transforms.ToPILImage()(images[n])
#print('PIL image size', imagePIL.size)
#imagePIL.save('test.png')
#display(imagePIL)

#display(Image.open('./data/rect'+str(n)+'.png'))

In [ ]:
import seaborn as sns
sns.set()
f = plt.subplot()
f.plot(range(len(losses_list)), losses_list)

plt.rcParams['figure.figsize'] = [30, 5]
xposition = np.array(range(num_epochs))  * (total_step) 
print(xposition)
for xc in xposition:
    plt.axvline(x=xc, color='k', linestyle='--')

In [ ]:
%matplotlib inline
#im = Image.open("Mew.jpg")
n = 5

image = io.imread('./data/rect'+str(n)+'.png')
image_tensor = torch.FloatTensor(image).permute(2, 0, 1) #PIL and torch expect difft orders
coords = torch.FloatTensor(true_coords[n])
with torch.no_grad():
    output = model(image_tensor)

print(true_coords[n])
print(output)
x,y = output.numpy().flatten()
imshow_coord(image, (x,y))
plt.imshow(image)



In [ ]:
#img = Image.new('RGB', (300,400), 'gray')
n = 0
#imshow_coord(images[n], outputs[n], labels[n])

print(images.size())
a = torchvision.utils.make_grid(images)
print(a.max(), a.min(), a.size())
#a = a / 2 + 0.5     # unnormalize

a = a.numpy()
a = np.transpose(a,(1,2,0))
print(a.shape)
print(a.ndim)

#ran = a.max() - a.min()
#a = (a/ran )

plt.imshow(a)
#x,y = labels[0].numpy()
#plt.scatter(x,y, color='g', marker='x', linewidth='1')

plt.rcParams['figure.figsize'] = [30, 5]
for i in range(len(labels)):
    x,y = labels[i].numpy()
    plt.scatter(x + i*IMG_X,y, color='g', marker='x', linewidth='1')
#plt.imshow(np.transpose(a, (2,0,1)))